In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from bs4 import BeautifulSoup

from datasets import DatasetDict, Dataset, load_dataset
from sklearn.model_selection import train_test_split
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# We can see all columns in df.head() / and .tail()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 1000) 
# pd.set_option('display.max_colwidth', None)

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


dataset_list = ['train', 'val', 'test']


train_dataid = '195XQ6oWYJE0b5Y4dXgUWOwtXOCPU5lIw'
val_dataid = '1vNIzBGt5KT4C4HPmmShZvSuN3rvtaukY'
test_dataid = '1IYqzvfY5JkRtCn3qE6yREOFf5bWZZ6f3'

download = drive.CreateFile({'id': train_dataid})
download.GetContentFile('train_data.csv')

download = drive.CreateFile({'id': val_dataid})
download.GetContentFile('val_data.csv')

download = drive.CreateFile({'id': test_dataid})
download.GetContentFile('test_data.csv')

In [ ]:
train_data = pd.read_csv('train_data.csv')
train_data.rename(columns={'answer':'answers'},inplace=True)
print(train_data.shape)
train_data.head()

(5615, 5)


,id,title,context,question,answer
0,0,how-to-make-halal-vanilla-extract,Step 1: Ingredients Halal Vanilla Extract: 3 u...,What is the step after Vegetable Glycerin and ...,{'text': 'Vanilla Beans Can Use With Vegetable...
1,1,how-to-make-cold-borscht,Step 1: Gathering Supplys: There are many vari...,What is the step before Time to Add the Beet J...,"{'text': 'Into the Blender.', 'answer_start': ..."
2,2,how-to-make-cold-borscht,Step 1: Gathering Supplys: There are many vari...,What is the step before Time to Add the Beet J...,"{'text': 'Open and Drain.', 'answer_start': [5..."
3,3,easy-handmade-pumpkin-puree,Step 1: Ingredients and Supplies: All you need...,What is the step after Cut the Pumpkin in Half...,"{'text': 'Steam in the Oven', 'answer_start': ..."
4,4,purple-gnocchi-with-gorgonzola,Step 1: Ingredients and Tools: This recipe req...,What is the step before Shape the Gnocchi?,"{'text': 'Prepare the Dough', 'answer_start': ..."


In [ ]:
test_data = pd.read_csv('test_data.csv')
test_data.rename(columns={'answer':'answers'},inplace=True)
print(test_data.shape)
test_data.head()

(687, 5)


,id,title,context,question,answer
0,0,drum-bbq-smoker-no-welding,Step 1: Prep: First I sanded off the paint on ...,What is the step after Making the Chimney?,"{'text': 'Grill Rack and Chimney', 'answer_sta..."
1,1,drum-bbq-smoker-no-welding,Step 1: Prep: First I sanded off the paint on ...,What is the step before Add Hinges?,"{'text': 'Cut Out the Lid', 'answer_start': [1..."
2,2,mexican-sweet-bread-pan-dulce,Step 1: Ingredients: This recipe makes 24 swee...,What is the step after Ingredients and before ...,"{'text': 'Knead the Dough', 'answer_start': [8..."
3,3,mexican-sweet-bread-pan-dulce,Step 1: Ingredients: This recipe makes 24 swee...,What is the step before Put the Bread in Muffi...,"{'text': 'Make the Topping', 'answer_start': [..."
4,4,holy-guacachurri,Step 1: Get All Your Ingredients and Utensils ...,What is the step after Get All Your Ingredient...,"{'text': 'Caramelize the Strawberries', 'answe..."


In [ ]:
val_data = pd.read_csv('val_data.csv')
val_data.rename(columns={'answer':'answers'},inplace=True)
print(val_data.shape)
val_data.head()

(648, 5)


,id,title,context,question,answer
0,0,bacon-pancake-poppers,Step 1: You Will Need...: \n -Bacon (...,What is the step after Jelly (Optional)?,"{'text': 'Deep Frying', 'answer_start': [1904]}"
1,1,baked-chicken-and-butternut-squash-tagine,Step 1: Ingredients!: You will need:\n-1 whole...,What is the step after Assemble the Dish?,"{'text': 'Bake', 'answer_start': [2111]}"
2,2,baby-groot-cupcakes,Step 1: Gingerbread Mix: Follow directions on ...,What is the step after Groot Assemble!!?,"{'text': 'Frosting', 'answer_start': [1947]}"
3,3,baby-groot-cupcakes,Step 1: Gingerbread Mix: Follow directions on ...,What is the step before Bake Your Cupcakes and...,"{'text': 'Gingerbread Mix', 'answer_start': [8]}"
4,4,baby-groot-cupcakes,Step 1: Gingerbread Mix: Follow directions on ...,What is the step after Bake Your Cookies and b...,"{'text': 'Groot Assemble!!', 'answer_start': [..."


In [ ]:
train = Dataset.from_pandas(train_data)
test = Dataset.from_pandas(test_data)
validation = Dataset.from_pandas(val_data)
final_dataset = DatasetDict({'train': train, 'validation': validation,'test': test})

In [ ]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answer'],
        num_rows: 5615
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answer'],
        num_rows: 648
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answer'],
        num_rows: 687
    })
})